In [9]:
import os
import shutil

import pandas as pd
import numpy as np

from os import listdir
from os.path import isfile, join

import plotly.figure_factory as ff

In [10]:
experiment_id = "NS-23.0061"

metadata_subdir = os.path.expanduser("~/fht.samba.data/experiments/ATACseq/{}/analysis/metadata/".format(experiment_id))
print('metadata_subdir exists:{}\n'.format(os.path.exists(metadata_subdir)))

bed_dir = '../beds/'
print('bed_dir exists:{}\n'.format(os.path.exists(bed_dir)))

genome = 'hg38' # or mm10

metadata_subdir exists:True

bed_dir exists:True



In [11]:
def create_dir(name_dir):
    # if os.path.exists(name_dir):
    #     shutil.rmtree(name_dir)
    # os.mkdir(name_dir)
    return name_dir

In [12]:
Motifs_dir = create_dir("../Motifs/")
print(Motifs_dir)

../Motifs/


In [13]:
def read_contrast(metadata_subdir, experiment_id): 
    
    metadata_files = [f for f in listdir(metadata_subdir) if isfile(join(metadata_subdir, f))]
    print('Files in metadata subdirectory:\n{}\n'.format(metadata_files))

    input_orig_contrast_dir = list(filter(lambda x: ('_contrasts_r' in x and experiment_id in x), metadata_files))[0] 
    contrasts_df = pd.read_table(metadata_subdir + input_orig_contrast_dir, delimiter = '\t', index_col = 0)
    print('input_orig_contrast_dir: {} - shape: {} \n'.format(input_orig_contrast_dir, contrasts_df.shape))  
    
    return contrasts_df

contrasts_df = read_contrast(metadata_subdir, experiment_id)
contrasts_df

Files in metadata subdirectory:
['ATACseq_ARID1AKO_ARID1BKD.csv', 'ATACseq_ARID1AKO_ARID1BKD.txt', 'NS-23.0061_metadata_r12x34.txt', 'ATACseq_ARID1AKO_ARID1BKD.xlsx', 'NS-23.0061_group_dict.json', 'NS-23.0061_contrast_dict.json', 'NS-23.0061_contrasts_r6x3.txt']

input_orig_contrast_dir: NS-23.0061_contrasts_r6x3.txt - shape: (6, 3) 



,test_group,negative_ctrl_grp,file_friendly_name
contrast,,,
ARID1BKD_HCT116 - WT_HCT116,ARID1BKD_HCT116,WT_HCT116,ARID1BKD_HCT116_WT_HCT116
ARID1AKO_HCT116 - WT_HCT116,ARID1AKO_HCT116,WT_HCT116,ARID1AKO_HCT116_WT_HCT116
ARID1AKO_ARID1BKD_HCT116 - WT_HCT116,ARID1AKO_ARID1BKD_HCT116,WT_HCT116,ARID1AKO_ARID1BKD_HCT116_WT_HCT116
ARID1AKO_ARID1BKD_HCT116 - ARID1AKO_HCT116,ARID1AKO_ARID1BKD_HCT116,ARID1AKO_HCT116,ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116
ARID1AKO_ARID1BKD_HCT116 - ARID1BKD_HCT116,ARID1AKO_ARID1BKD_HCT116,ARID1BKD_HCT116,ARID1AKO_ARID1BKD_HCT116_ARID1BKD_HCT116
ARID1BKD_TOV21G - WT_TOV21G,ARID1BKD_TOV21G,WT_TOV21G,ARID1BKD_TOV21G_WT_TOV21G


# Run HOMER

In [ ]:
def read_motifs(contrasts_df, bed_dir):
    
    for contrast in contrasts_df['file_friendly_name'][:]:

        print("\ncontrast:  {}".format(contrast))
        
        # down
        !findMotifsGenome.pl  {bed_dir}ATAC_seq_{contrast}_down.bed   {genome}   {Motifs_dir}ATAC_seq_{contrast}_down -bg  {bed_dir}ATAC_seq_{contrast}_unchanged.bed

        # up
        !findMotifsGenome.pl  {bed_dir}ATAC_seq_{contrast}_up.bed   {genome}  {Motifs_dir}ATAC_seq_{contrast}_up -bg  {bed_dir}ATAC_seq_{contrast}_unchanged.bed

read_motifs(contrasts_df, bed_dir)

# Build output table as pdf

In [14]:
def build_contrast_table(contrast, Motifs_dir, direction):
    
        motif_results_path = Motifs_dir + 'ATAC_seq_' + contrast + '_' + direction + '/knownResults.txt'
        # print("\nmotif_results_path:  {}\n".format(motif_results_path))
        
        motif_df = pd.read_table(motif_results_path, delimiter = '\t',  index_col = 0)[ 1:11][["Consensus", "P-value"]]
        ### TO CHANGE ###
        # test_group = list(contrasts_df[contrasts_df['file_friendly_name'] == contrast].test_group)[0]
        test_group = list(contrasts_df[contrasts_df['file_friendly_name'] == contrast].file_friendly_name)[0]

        motif_df[test_group] = motif_df.index.str.split(pat='(', expand=False).str[0]
        motif_df[' '] = '           ' 
        motif_df = motif_df.reset_index(drop=True)[[test_group,  'P-value', ' ' ]]
        motif_df[' '] = motif_df['P-value'].map(lambda x: '{:.1e}'.format(x)) 
        return motif_df[[test_group, " "]]

In [15]:
def build_output_table(contrasts_df, Motifs_dir, direction):
    print('direction: {}'.format(direction))
    motifs_output_table = pd.DataFrame(index=range(10))
    for contrast in contrasts_df['file_friendly_name'][:]:
        print("\n{}".format(contrast))
        motifs_df = build_contrast_table(contrast, Motifs_dir, direction)
        motifs_output_table = pd.concat([motifs_output_table, motifs_df], axis =1 )  
    display(motifs_output_table.head(n=15))  
    filename = Motifs_dir + "Motifs_" + direction + ".pdf"
    print('filename is : {}'.format(filename))
        
    fig = ff.create_table(motifs_output_table)
    fig.update_layout(
    autosize=False,
    width=5000,
    height=2000, font=dict(size=32)
    )
    # fig.show()

    fig.write_image(filename, scale=2)

In [16]:
build_output_table(contrasts_df, Motifs_dir, direction = 'down')
build_output_table(contrasts_df, Motifs_dir, direction = 'up')

direction: down

ARID1BKD_HCT116_WT_HCT116

ARID1AKO_HCT116_WT_HCT116

ARID1AKO_ARID1BKD_HCT116_WT_HCT116

ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116

ARID1AKO_ARID1BKD_HCT116_ARID1BKD_HCT116

ARID1BKD_TOV21G_WT_TOV21G


,ARID1BKD_HCT116_WT_HCT116,,ARID1AKO_HCT116_WT_HCT116,,ARID1AKO_ARID1BKD_HCT116_WT_HCT116,,ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116,,ARID1AKO_ARID1BKD_HCT116_ARID1BKD_HCT116,,ARID1BKD_TOV21G_WT_TOV21G,
0,RUNX,1.0e-39,Atf3,0.0e+00,Fos,0.0e+00,Atf3,0.0e+00,Fos,0.0e+00,Atf3,1.0e-22
1,RUNX2,1.0e-34,Fos,0.0e+00,Atf3,0.0e+00,Fra1,0.0e+00,Atf3,0.0e+00,Fra1,1.0e-21
2,RUNX-AML,1.0e-32,BATF,0.0e+00,BATF,0.0e+00,BATF,0.0e+00,JunB,0.0e+00,BATF,1.0e-21
3,TEAD4,1.0e-11,Fra2,0.0e+00,Fra2,0.0e+00,JunB,0.0e+00,BATF,0.0e+00,JunB,1.0e-21
4,TEAD,1.0e-10,JunB,0.0e+00,JunB,0.0e+00,AP-1,0.0e+00,Fra2,0.0e+00,AP-1,1.0e-20
5,Tcf7,1.0e-09,AP-1,0.0e+00,AP-1,0.0e+00,Fra2,0.0e+00,AP-1,0.0e+00,Fra2,1.0e-20
6,TEAD1,1.0e-09,Fosl2,0.0e+00,Fosl2,0.0e+00,Fosl2,0.0e+00,Fosl2,0.0e+00,Fosl2,1.0e-20
7,TEAD2,1.0e-08,Jun-AP1,0.0e+00,Jun-AP1,0.0e+00,Jun-AP1,0.0e+00,Jun-AP1,0.0e+00,Jun-AP1,1.0e-15
8,LEF1,1.0e-07,TEAD1,1.0e-244,Bach2,0.0e+00,Bach2,1.0e-177,Bach2,0.0e+00,Bach2,1.0e-09
9,TEAD3,1.0e-07,TEAD3,1.0e-240,TEAD1,1.0e-306,RUNX1,1.0e-77,TEAD1,1.0e-219,PAX6,1.0e-07


filename is : ../Motifs/Motifs_down.pdf
direction: up

ARID1BKD_HCT116_WT_HCT116

ARID1AKO_HCT116_WT_HCT116

ARID1AKO_ARID1BKD_HCT116_WT_HCT116

ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116

ARID1AKO_ARID1BKD_HCT116_ARID1BKD_HCT116

ARID1BKD_TOV21G_WT_TOV21G


,ARID1BKD_HCT116_WT_HCT116,,ARID1AKO_HCT116_WT_HCT116,,ARID1AKO_ARID1BKD_HCT116_WT_HCT116,,ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116,,ARID1AKO_ARID1BKD_HCT116_ARID1BKD_HCT116,,ARID1BKD_TOV21G_WT_TOV21G,
0,CEBP:CEBP,1.0e-03,Sp1,1.0e-06,NFY,1.0e-18,ZBTB18,1.0e-02,Sp1,1.0e-23,Six2,1.0e-02
1,SF1,1.0e-03,NFY,1.0e-05,KLF3,1.0e-15,PBX1,1.0e-02,KLF3,1.0e-18,Meis1,1.0e-02
2,Nr5a2,1.0e-03,KLF1,1.0e-04,KLF1,1.0e-10,Phox2b,1.0e-02,Klf9,1.0e-13,Pitx1:Ebox,1.0e-02
3,NFAT,1.0e-02,DLX1,1.0e-04,Sp5,1.0e-10,Pbx3,1.0e-02,KLF1,1.0e-10,ZNF7,1.0e-02
4,Slug,1.0e-02,Sp5,1.0e-03,Klf9,1.0e-09,Phox2a,1.0e-02,KLF6,1.0e-08,LRF,1.0e-02
5,BMYB,1.0e-02,TATA-Box,1.0e-03,Sp2,1.0e-06,IRF4,1.0e-02,Sp5,1.0e-08,c-Myc,1.0e-01
6,Ptf1a,1.0e-02,Klf9,1.0e-03,KLF14,1.0e-06,Gata4,1.0e-02,Klf4,1.0e-07,Pbx3,1.0e-01
7,HEB,1.0e-02,Sp2,1.0e-03,KLF5,1.0e-06,AR-halfsite,1.0e-02,KLF5,1.0e-06,Gli2,1.0e-01
8,KLF14,1.0e-02,KLF5,1.0e-02,Smad3,1.0e-05,Prop1,1.0e-02,Sp2,1.0e-06,ZNF692,1.0e-01
9,ZEB1,1.0e-02,YY1,1.0e-02,Maz,1.0e-04,Hoxa10,1.0e-01,Pbx3,1.0e-05,Sox7,1.0e-01


filename is : ../Motifs/Motifs_up.pdf


# Build motif heatmap

In [17]:
def build_contrast_table(contrast, Motifs_dir, direction):
    
        motif_results_path = Motifs_dir + 'ATAC_seq_' + contrast + '_' + direction + '/knownResults.txt'
        # print("\nmotif_results_path:  {}\n".format(motif_results_path))
        
        motif_df = pd.read_table(motif_results_path, delimiter = '\t',  index_col = 0)#[["P-value"]]
        # display(motif_df)
        ### TO CHANGE ###
        # test_group = list(contrasts_df[contrasts_df['file_friendly_name'] == contrast].test_group)[0]
        test_group = list(contrasts_df[contrasts_df['file_friendly_name'] == contrast].file_friendly_name)[0]

        motif_df['Motifs'] = motif_df.index.str.split(pat='(', expand=False).str[0]
        motif_df = motif_df.sort_values('Motifs',ascending=True)
        motif_df = motif_df.reset_index(drop=True)
        if direction == 'up':
            tmp = motif_df['Log P-value'] / np.log(10)
            tmp[tmp  > -1] = -1
            motif_df[test_group] = np.log10( -tmp)
        else:
            tmp = motif_df['Log P-value'] / np.log(10)
            tmp[tmp  > -1] = -1
            motif_df[test_group] = (-1) * np.log10(-tmp)
        motif_df = motif_df[['Motifs', test_group]]
        
        return motif_df

In [18]:
def build_output_table_by_direction(contrasts_df, Motifs_dir, direction):
    print('direction: {}'.format(direction))
   
    motifs_output_table = build_contrast_table(contrasts_df['file_friendly_name'][0], Motifs_dir, direction)
    print('motifs_output_table saved at: shape: {} \n'.format(motifs_output_table.shape))
    display(motifs_output_table.head(n=2))  
    
    for contrast in contrasts_df['file_friendly_name'][1:]:
        # print("\n{}".format(contrast))
        
        motif_results_path = Motifs_dir + 'ATAC_seq_' + contrast + '_' + direction + '/knownResults.txt'
        if os.path.exists(motif_results_path):
            
            motifs_df = build_contrast_table(contrast, Motifs_dir, direction)
            # print('motifs_df saved at: shape: {} \n'.format(motifs_df.shape))
            # display(motifs_df.head(n=2)) 
            
            motifs_output_table = pd.concat([motifs_output_table, motifs_df.drop(['Motifs'],axis=1)], axis =1)
            
    motifs_output_table.set_index('Motifs', inplace=True)
    motifs_output_table.index = [direction + '_' + str(idx) for idx in motifs_output_table.index]
    motifs_output_table = motifs_output_table.groupby(motifs_output_table.index).mean()

    print('\nmotifs_output_table saved at: shape: {} \n'.format(motifs_output_table.shape))
    display(motifs_output_table.head(n=2))     
    return motifs_output_table

In [19]:
def build_output_table(contrasts_df, Motifs_dir):
       
    all_motifs_up = build_output_table_by_direction(contrasts_df, Motifs_dir, direction = 'up')
    all_motifs_down = build_output_table_by_direction(contrasts_df, Motifs_dir, direction = 'down')

    all_motifs = pd.concat([all_motifs_down, all_motifs_up]) 
    filename = "{}Motif_heatmap_HOMER_{}.csv".format(Motifs_dir, experiment_id)
    print('\nall_motifs saved at: {} - shape: {} \n'.format(filename, all_motifs.shape))  
    all_motifs.to_csv(filename)

    display(all_motifs.head(2))
    return all_motifs
    
all_motifs = build_output_table(contrasts_df, Motifs_dir)

direction: up
motifs_output_table saved at: shape: (440, 2) 



,Motifs,ARID1BKD_HCT116_WT_HCT116
0,AMYB,0.082765
1,AP-1,0.000000



motifs_output_table saved at: shape: (419, 6) 



,ARID1BKD_HCT116_WT_HCT116,ARID1AKO_HCT116_WT_HCT116,ARID1AKO_ARID1BKD_HCT116_WT_HCT116,ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116,ARID1AKO_ARID1BKD_HCT116_ARID1BKD_HCT116,ARID1BKD_TOV21G_WT_TOV21G
up_AMYB,0.082765,0.0,0.345695,0.0,0.036419,0.0
up_AP-1,0.000000,0.0,0.000000,0.0,0.000000,0.0


direction: down
motifs_output_table saved at: shape: (440, 2) 



,Motifs,ARID1BKD_HCT116_WT_HCT116
0,AMYB,-0.0
1,AP-1,-0.0



motifs_output_table saved at: shape: (419, 6) 



,ARID1BKD_HCT116_WT_HCT116,ARID1AKO_HCT116_WT_HCT116,ARID1AKO_ARID1BKD_HCT116_WT_HCT116,ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116,ARID1AKO_ARID1BKD_HCT116_ARID1BKD_HCT116,ARID1BKD_TOV21G_WT_TOV21G
down_AMYB,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
down_AP-1,0.0,-2.841089,-3.276972,-2.755718,-3.261654,-1.310805



all_motifs saved at: ../Motifs/Motif_heatmap_HOMER_NS-23.0061.csv - shape: (838, 6) 



,ARID1BKD_HCT116_WT_HCT116,ARID1AKO_HCT116_WT_HCT116,ARID1AKO_ARID1BKD_HCT116_WT_HCT116,ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116,ARID1AKO_ARID1BKD_HCT116_ARID1BKD_HCT116,ARID1BKD_TOV21G_WT_TOV21G
down_AMYB,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
down_AP-1,0.0,-2.841089,-3.276972,-2.755718,-3.261654,-1.310805


In [20]:
print('min: {}'.format(all_motifs.min().min()))
print('max: {}'.format(all_motifs.max().max()))

min: -3.307658813790339
max: 1.521104989683512


In [21]:
all_motifs.values.flatten()

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
import plotly.express as px
# df = px.data.tips()
fig = px.ecdf(all_motifs.values.flatten())
fig.show()

In [ ]:
np.isnan(all_motifs.values.flatten()).all()

# Scatter plot

In [ ]:
import plotly.express as px
def build_output_table(contrasts_df, Motifs_dir, direction):
   
    for contrast in contrasts_df['file_friendly_name'][:]:

        print("\ncontrast:  {}".format(contrast))
        ### TO CHANGE ###
        # test_group = list(contrasts_df[contrasts_df['file_friendly_name'] == contrast].test_group)[0]
        test_group = list(contrasts_df[contrasts_df['file_friendly_name'] == contrast].file_friendly_name)[0]
        motif_results_path = Motifs_dir + 'ATAC_seq_' + contrast + '_' + direction + '/knownResults.txt'
        print("\nmotif_results_path:  {}\n".format(motif_results_path)) 
        if os.path.exists(motif_results_path):
        
            ## READ HOMER RESULTS ##
            motif_df = pd.read_table(motif_results_path, delimiter = '\t',  index_col = 0)
            motif_df['Motifs'] = motif_df.index.str.split(pat='(', expand=False).str[0]

            ## REFORMAT % OF SEQUENCES ## 
            motif_df['% of Target Sequences with Motif'] = pd.to_numeric(motif_df['% of Target Sequences with Motif'].replace('[%,]', '',regex=True) , errors='coerce')
            motif_df['% of Background Sequences with Motif'] = pd.to_numeric(motif_df['% of Background Sequences with Motif'].replace('[%,]', '',regex=True) , errors='coerce')

            ## COMPUTE RATIO AND P-VALUE ##
            motif_df['%Target/%Background'] = motif_df['% of Target Sequences with Motif'] / motif_df['% of Background Sequences with Motif']      
            motif_df['-log10(P-value)'] = (-1) * np.log10( motif_df['P-value'] )
            motif_df['-log10(P-Value)'] = (-1) * motif_df['Log P-value']   
                        
            ## ADD TF FAMILY ##
            motif_df['TF'] = np.where(motif_df['Motifs'].str.contains('AP1|AP-1|Fosl1|Fra1|Fosl2|Fra2|Jun|JunB|Fos|BATF|ATF|Atf3'), 'AP1 Family', 'others')
            motif_df.sort_values(['TF',"%Target/%Background"], inplace=True, ascending = False)
            # display(motif_df)
            
            ## CREATE OTHER DF WITH ONLY TF FAMILY TO ADD POINTS OVER OTHERS ##
            AP1_family = motif_df.loc[motif_df['TF'] == 'AP1 Family', ['%Target/%Background', '-log10(P-Value)', 'Motifs']]
            # AP1_family = AP1_family.loc[AP1_family['%Target/%Background'] > 2 ]
            AP1_family = AP1_family.loc[AP1_family['-log10(P-Value)'] > 100 ]
            
            ## OUTPUT PLOT ##
            fig = px.scatter(motif_df, x= '%Target/%Background', y='-log10(P-Value)', color='TF',
                             hover_data = ['Motifs'],
                             title = test_group + ' - ' + direction,
                             color_discrete_map= {'AP1 Family': 'tomato',
                                                  'others': 'dimgray',
                                          })  #https://stackoverflow.com/questions/72496150/user-friendly-names-for-plotly-css-colors
            fig.update_layout(
                # autosize=True, width=1000, height=500,
            margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="LightSteelBlue")  
            fig.update_traces(marker={'size': 13})

            # ## TO ADD TEXT OF TF NEXT TO POINT ##
            # for i, (x,y,z) in enumerate(AP1_family.itertuples(index=False)):
            #     ytextposition = -20 if i%2==0 else 10
            #     xtextposition = -20 if i%2==0 else 10
            #     fig.add_annotation(
            #         x=x, y=y,
            #         text=z,
            #         showarrow=False,
            #         yshift=ytextposition,
            #         xshift=xtextposition
            #     )
            # fig.show()            
            
            ## SAVE PLOT ##
            filename = Motifs_dir + 'ATAC_seq_' + contrast + '_' + direction + '/' + test_group + '_' + direction + ".html"
            print(filename)
            fig.write_html(filename)

In [ ]:
build_output_table(contrasts_df, Motifs_dir,  direction= 'down')

In [ ]:
build_output_table(contrasts_df, Motifs_dir,  direction= 'up')